In [63]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.linalg import eigh
import qutip as qt
import sympy as sym
from joblib import Parallel, delayed, cpu_count

%matplotlib ipympl
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Palatino"],
# })
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})

In [64]:
fF  = 1e-15
nH  = 1e-9
GHz = 1e9

In [65]:
nmax_r = 10
nmax_f = 15

only_inner = True
compensate_extra_cap = False

In [66]:
CR_1, CF_1, LF_1, LR_1, EJ_1, Δ_1, ω_r_1 = sq_ext.get_experimental_parameters('qubit_1')
CR_2, CF_2, LF_2, LR_2, EJ_2, Δ_2, ω_r_2 = sq_ext.get_experimental_parameters('qubit_2')
CR_3, CF_3, LF_3, LR_3, EJ_3, Δ_3, ω_r_3 = sq_ext.get_experimental_parameters('qubit_3')

## Single qubit

In [78]:
## Testings
Δ=0
fluxonium = sq_ext.sq_fluxonium(C_F_eff=CF_1, L_F_eff=LF_1, EJ=EJ_1, Δ=Δ_1, nmax_f=nmax_f)
resonator = sq_ext.sq_resonator(C_R_eff=CR_1, L_R_eff=LR_1, Δ=Δ_1, nmax_f=nmax_f)

H_0 = sq_ext.hamiltonian_qubit(fluxonium = fluxonium, resonator = resonator, Δ=0) 
H   = sq_ext.hamiltonian_qubit(fluxonium = fluxonium, resonator = resonator, Δ=Δ) 

H1 = fluxonium.hamiltonian()
H2 = resonator.hamiltonian()

ψ_0_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2,solver='numpy')[1][:,0])
ψ_1_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2,solver='numpy')[1][:,1])
ψ_0_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2,solver='numpy')[1][:,0])
ψ_1_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2,solver='numpy')[1][:,1])

ψ_0 = [qt.tensor(ψ_0_f, ψ_0_r), qt.tensor(ψ_1_f,ψ_0_r), qt.tensor(ψ_0_f,ψ_1_r), qt.tensor(ψ_1_f,ψ_1_r)]

In [75]:
ψ_0 = None

In [79]:
# def H_eff_SWT(H_0, H, n_eig, out='GHz', real=False, remove_ground=False, return_transformation=False,ψ_0=None ):
n_eig = 6
out='GHz'
if ψ_0 is None:
    ψ_0 = sq_ext.diag(H_0, n_eig, real=True, solver='numpy')[1]
    E, ψ = sq_ext.diag(H, n_eig, real=False, solver='scipy', out=out)

    Q = np.zeros((n_eig, n_eig), dtype=complex)
    for i in range(n_eig):
        for j in range(n_eig):
            Q[i, j] = ψ_0[:,i].conj().T @ ψ[:,j]
else:
    n_eig=len(ψ_0)
    E, ψ = sq_ext.diag(H, n_eig, real=False, solver='Qutip', out=out, qObj=True)

    Q = np.zeros((n_eig, n_eig), dtype=complex)
    for i in range(n_eig):
        for j in range(n_eig):
            Q[i, j] = (ψ_0[i].dag() * ψ[j]).data[0,0]


U, s, Vh = np.linalg.svd(Q)
A = U @ Vh

H_eff_SWT = A @ np.diag(E) @ A.T.conj()

In [77]:
Nf, Nr = sq_ext.sq_get_energy_indices_hamiltonian(H_qubit = H_0, H_fluxonium=H1, H_resonator=H2, n_eig=5)
subspace_indices = np.intersect1d(np.where(Nf<=1), np.where(Nr<=1)).tolist()

H_eff_SWT_subspace  = H_eff_SWT [subspace_indices][:, subspace_indices]
P = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT_subspace, print_pretty=True)

 9.885596502069895	*	 I ⨂I 
 -1.8443489399068576	*	 I ⨂σz 
 -3.229999999999823	*	 σz ⨂I 


In [80]:
sq_ext.decomposition_in_pauli_4x4(H_eff_SWT, print_pretty=True);

 9.372109825309492	*	 I ⨂I 
 -1.3308622631464644	*	 I ⨂σz 
 -2.7165133232394267	*	 σz ⨂I 
 -0.5134866767604	*	 σz ⨂σz 


In [82]:
sq_ext.decomposition_in_pauli_4x4(sq_ext.H_eff_SWT(H_0, H, n_eig, ψ_0=ψ_0), print_pretty=True);

 9.372109825309533	*	 I ⨂I 
 -1.3308622631464646	*	 I ⨂σz 
 -2.7165133232394796	*	 σz ⨂I 
 -0.5134866767603896	*	 σz ⨂σz 
